In [28]:
%load_ext autoreload
%autoreload 2

from util_0704 import *
from myalgorithm_0704_1 import algorithm

import statsmodels.api as sm
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


- 실험해볼 내용들

1. 초기 최적화 종료 후 인접한 번들끼리 원소를 교환해보면 어떨까?
2. 인접한 2번들1과 2번들2가 있을 때 3번들1, 1번들1로 바꾼 후에 다시 1번들을 묶어보면 어떨까? 

## 기본 코드

In [77]:
problem_file = '../alg_test_problems_20240429/TEST_K200_1.json'

In [127]:
with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

In [138]:
## ------------------- 초기 상태 할당 코드 -------------------------

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

## --------------- 초기 번들링 최적화 코드 --------------------------

inf = float('inf')

car_rider = [rider for rider in ALL_RIDERS if rider.type == 'CAR'][0]
bike_rider = [rider for rider in ALL_RIDERS if rider.type == 'BIKE'][0]
walk_rider = [rider for rider in ALL_RIDERS if rider.type == 'WALK'][0]

init_availables = [rider.available_number for rider in ALL_RIDERS]

min_init_cost = inf
min_init_cost_bundle = []
min_init_cost_rider_availables = []

weight1 = 1
weight2 = -1.5

all_bundles, rider_availables, cost = get_init_bundle_4_order_bundle_prefered_with_reassigning_riders(K, ALL_RIDERS, ALL_ORDERS, DIST, init_availables, weight1, weight2, get_dist_by_coords)
for rider_i in range(3):
    ALL_RIDERS[rider_i].available_number = rider_availables[rider_i]

## -------------- 솔루션 제작 및 실현 가능성 확인 코드 ---------------- 

solution = [
        # rider type, shop_seq, dlv_seq
        [bundle.rider.type, bundle.shop_seq, bundle.dlv_seq]
        for bundle in all_bundles
]

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, solution)

print(checked_solution['avg_cost'])

3413.8435000000004


### 초기 최적화 종료 후 인접한 번들끼리 원소를 교환해보기

In [80]:
all_bundles_len = len(all_bundles)

G = [[] * all_bundles_len for _ in range(all_bundles_len)]

for bundle_i in range(all_bundles_len):
    bundle = all_bundles[bundle_i]

    shop_seq = bundle.shop_seq

    xs_s_sum = 0
    ys_s_sum = 0

    xs_e_sum = 0
    ys_e_sum = 0

    readytimes_sum = 0
    deadlines_sum = 0

    shop_seq_len = len(shop_seq)

    for order_num in shop_seq:
        order = ALL_ORDERS[order_num]

        xs_s_sum += order.shop_lat
        ys_s_sum += order.shop_lon

        xs_e_sum += order.dlv_lat
        ys_e_sum += order.dlv_lon

        readytimes_sum += order.ready_time
        deadlines_sum += order.deadline

    xs_s_avg = xs_s_sum / shop_seq_len
    ys_s_avg = ys_s_sum / shop_seq_len

    xs_e_avg = xs_e_sum / shop_seq_len
    ys_e_avg = ys_e_sum / shop_seq_len

    readytimes_avg = readytimes_sum / shop_seq_len
    deadlines_avg = deadlines_sum / shop_seq_len

    avg_info = [xs_s_avg, ys_s_avg, xs_e_avg, ys_e_avg, readytimes_avg, deadlines_avg]

    bundle.avg_info = avg_info

edges = []
for i in range(len(all_bundles)):
    for j in range(i + 1, len(all_bundles)):
        avg_info1 = all_bundles[i].avg_info
        avg_info2 = all_bundles[j].avg_info

        sx1, sy1, ex1, ey1, r1, d1 = avg_info1
        sx2, sy2, ex2, ey2, r2, d2 = avg_info2

        r_diff = abs(r1 - r2)
        d_diff = abs(d1 - d2)

        start_end_diff = get_dist_by_coords((sx1 + sx2) / 2, (sy1 + sy2) / 2, (ex1 + ex2) / 2, (ey1 + ey2) / 2)

        dist1 = get_dist_by_coords(sx1, sy1, sx2, sy2)
        dist2 = get_dist_by_coords(ex1, ey1, ex2, ey2)

        diff_score = dist1 + dist2 + r_diff * weight1 + d_diff * weight1 + start_end_diff * weight2

        edges.append((i, j, diff_score))

edges.sort(key=lambda x: x[2])

In [81]:
for bundle_i1, bundle_i2, _ in edges:
    G[bundle_i1].append(bundle_i2)
    G[bundle_i2].append(bundle_i1)

In [82]:
for bundle_i1 in range(all_bundles_len):
    G[bundle_i1] = G[bundle_i1][:5]

In [83]:
for _ in range(100000):
    bundle_i1 = random.randint(0, all_bundles_len - 1)
    bundle_i2_nth = len(G[bundle_i1]) - 1
    bundle_i2 = G[bundle_i1][bundle_i2_nth]

    bundle1 = all_bundles[bundle_i1]
    bundle2 = all_bundles[bundle_i2]

    shop_seq1 = bundle1.shop_seq
    shop_seq2 = bundle2.shop_seq

    ex_1_to_2_i = random.randint(0, len(bundle1.shop_seq) - 1)
    ex_2_to_1_i = random.randint(0, len(bundle2.shop_seq) - 1)

    ex_1_to_2 = shop_seq1[ex_1_to_2_i]
    ex_2_to_1 = shop_seq2[ex_2_to_1_i]

    new_shop_seq1 = shop_seq1.copy()
    new_shop_seq2 = shop_seq2.copy()

    new_shop_seq1.remove(ex_1_to_2)
    new_shop_seq1.append(ex_2_to_1)
    new_shop_seq2.remove(ex_2_to_1)
    new_shop_seq2.append(ex_1_to_2)

    rider1 = bundle1.rider
    rider2 = bundle2.rider

    new_bundle1 = try_feasible_route(K, DIST, ALL_ORDERS, ALL_RIDERS, new_shop_seq1, rider1)
    new_bundle2 = try_feasible_route(K, DIST, ALL_ORDERS, ALL_RIDERS, new_shop_seq2, rider2)

    if new_bundle1 and new_bundle2:
        old_cost = bundle1.cost + bundle2.cost
        new_cost = new_bundle1.cost + new_bundle2.cost

        if new_cost < old_cost:
            print('exchanged', end=' ')

            all_bundles[bundle_i1] = new_bundle1
            all_bundles[bundle_i2] = new_bundle2

exchanged exchanged 

> 결과가 개선되는 경우가 거의 없음

### 인접한 2번들1과 2번들2가 있을 때 3번들1, 1번들1로 바꾼 후에 다시 1번들을 묶어보기

In [94]:
two_order_bundles = [v for v in all_bundles if len(v.shop_seq) == 2]
other_length_bundles = [v for v in all_bundles if len(v.shop_seq) != 2]

two_order_bundles_len = len(two_order_bundles)

G = [[] * two_order_bundles_len for _ in range(two_order_bundles_len)]

for bundle_i in range(two_order_bundles_len):
    bundle = two_order_bundles[bundle_i]

    shop_seq = bundle.shop_seq

    xs_s_sum = 0
    ys_s_sum = 0

    xs_e_sum = 0
    ys_e_sum = 0

    readytimes_sum = 0
    deadlines_sum = 0

    shop_seq_len = len(shop_seq)

    for order_num in shop_seq:
        order = ALL_ORDERS[order_num]

        xs_s_sum += order.shop_lat
        ys_s_sum += order.shop_lon

        xs_e_sum += order.dlv_lat
        ys_e_sum += order.dlv_lon

        readytimes_sum += order.ready_time
        deadlines_sum += order.deadline

    xs_s_avg = xs_s_sum / shop_seq_len
    ys_s_avg = ys_s_sum / shop_seq_len

    xs_e_avg = xs_e_sum / shop_seq_len
    ys_e_avg = ys_e_sum / shop_seq_len

    readytimes_avg = readytimes_sum / shop_seq_len
    deadlines_avg = deadlines_sum / shop_seq_len

    avg_info = [xs_s_avg, ys_s_avg, xs_e_avg, ys_e_avg, readytimes_avg, deadlines_avg]

    bundle.avg_info = avg_info

edges = []
for i in range(len(two_order_bundles)):
    for j in range(i + 1, len(two_order_bundles)):
        avg_info1 = two_order_bundles[i].avg_info
        avg_info2 = two_order_bundles[j].avg_info

        sx1, sy1, ex1, ey1, r1, d1 = avg_info1
        sx2, sy2, ex2, ey2, r2, d2 = avg_info2

        r_diff = abs(r1 - r2)
        d_diff = abs(d1 - d2)

        start_end_diff = get_dist_by_coords((sx1 + sx2) / 2, (sy1 + sy2) / 2, (ex1 + ex2) / 2, (ey1 + ey2) / 2)

        dist1 = get_dist_by_coords(sx1, sy1, sx2, sy2)
        dist2 = get_dist_by_coords(ex1, ey1, ex2, ey2)

        diff_score = dist1 + dist2 + r_diff * weight1 + d_diff * weight1 + start_end_diff * weight2

        edges.append((i, j, diff_score))

edges.sort(key=lambda x: x[2])

#### 일부 검사

In [95]:
for bundle_i1, bundle_i2, _ in edges:
    G[bundle_i1].append(bundle_i2)
    G[bundle_i2].append(bundle_i1)

for i in range(two_order_bundles_len):
    G[i] = G[i][:4]

is_moved = [False] * two_order_bundles_len
newerly_made_bundles = []
for bundle_i1 in range(two_order_bundles_len):
    for bundle_i2 in G[bundle_i1]:
        if is_moved[bundle_i1] or is_moved[bundle_i2]:
            continue

        bundle1 = two_order_bundles[bundle_i1]
        bundle2 = two_order_bundles[bundle_i2]

        shop_seq1 = bundle1.shop_seq
        shop_seq2 = bundle2.shop_seq

        rider1 = bundle1.rider
        rider2 = bundle2.rider

        for i in range(2):
            new_shop_seq1 = shop_seq1.copy()
            new_shop_seq2 = shop_seq2.copy()

            new_shop_seq2.append(new_shop_seq1.pop(i))

            new_bundle1 = try_feasible_route(K, DIST, ALL_ORDERS, ALL_RIDERS, new_shop_seq1, rider1)
            new_bundle2 = try_feasible_route(K, DIST, ALL_ORDERS, ALL_RIDERS, new_shop_seq2, rider2)

            if new_bundle1 and new_bundle2:
                print('moved', end=' ')

                is_moved[bundle_i1] = is_moved[bundle_i2] = True
                newerly_made_bundles.append(new_bundle1)
                newerly_made_bundles.append(new_bundle2)
            

moved moved moved 

#### 전체 검사

In [132]:
for bundle_i1, bundle_i2, _ in edges:
    G[bundle_i1].append(bundle_i2)
    G[bundle_i2].append(bundle_i1)

is_moved = [False] * two_order_bundles_len
newerly_made_bundles = []
for j in range(two_order_bundles_len):
    for bundle_i1 in range(two_order_bundles_len):
        bundle_i2 = G[bundle_i1][j]
        
        if is_moved[bundle_i1] or is_moved[bundle_i2]:
            continue

        bundle1 = two_order_bundles[bundle_i1]
        bundle2 = two_order_bundles[bundle_i2]

        shop_seq1 = bundle1.shop_seq
        shop_seq2 = bundle2.shop_seq

        rider1 = bundle1.rider
        rider2 = bundle2.rider

        for i in range(2):
            new_shop_seq1 = shop_seq1.copy()
            new_shop_seq2 = shop_seq2.copy()

            new_shop_seq2.append(new_shop_seq1.pop(i))

            new_bundle1 = try_feasible_route(K, DIST, ALL_ORDERS, ALL_RIDERS, new_shop_seq1, rider1)
            new_bundle2 = try_feasible_route(K, DIST, ALL_ORDERS, ALL_RIDERS, new_shop_seq2, rider2)

            if new_bundle1 and new_bundle2:
                print('moved', end=' ')

                is_moved[bundle_i1] = is_moved[bundle_i2] = True
                newerly_made_bundles.append(new_bundle1)
                newerly_made_bundles.append(new_bundle2)
            

moved moved moved moved moved 

> 결과는 일부일 때에 비해 옮겨지는 경우가 늘어났음

In [133]:
new_two_order_bundles = [two_order_bundles[i] for i in range(two_order_bundles_len) if not is_moved[i]]

new_all_bundles = new_two_order_bundles + newerly_made_bundles + other_length_bundles

In [135]:
new_all_bundles2, result_availables = kruskal_bundling(K, DIST, ALL_ORDERS, ALL_RIDERS, 1, -1.5, try_merging_bundles_by_dist, 4, 'avg', new_all_bundles, get_dist_by_coords)
for rider_i in range(3):
    ALL_RIDERS[rider_i].available_number = result_availables[rider_i]

In [137]:
solution = [
        # rider type, shop_seq, dlv_seq
        [bundle.rider.type, bundle.shop_seq, bundle.dlv_seq]
        for bundle in new_all_bundles2
]

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, solution)

print(checked_solution['avg_cost'])

3399.8945000000003
